# Transition-based arc-eager unlabeled dependency parser for Ukrainian

## Read the data

Useful links:
* [UD corpus for Ukrainian](https://github.com/UniversalDependencies/UD_Ukrainian-IU/)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

with open("uk_iu-ud-train.conllu", "r") as f:
    data = f.read()

trees = parse(data)

In [2]:
tree = trees[0]
print(tree[0])

OrderedDict([('id', 1), ('form', 'У'), ('lemma', 'у'), ('upostag', 'ADP'), ('xpostag', 'Spsl'), ('feats', OrderedDict([('Case', 'Loc')])), ('head', 2), ('deprel', 'case'), ('deps', [('case', 2)]), ('misc', OrderedDict([('Id', '0003'), ('LTranslit', 'u'), ('Translit', 'U')]))])


In [3]:
for node in tree:
    head = node["head"]
    print("{} <-- {}".format(node["form"],
                             tree[head - 1]["form"]
                             if head > 0 else "root"))

У <-- домі
домі <-- була
римського <-- патриція
патриція <-- домі
Руфіна <-- патриція
була <-- root
прегарна <-- фреска
фреска <-- була
, <-- зображення
зображення <-- фреска
Венери <-- зображення
та <-- Адоніса
Адоніса <-- Венери
. <-- була


## Design actions and the oracle

We will be using a static oracle that reproduces a single valid order of actions.

In [4]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

In [5]:
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [6]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations = [ROOT], tree[:], []
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            print("Relations:", relations)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT_0']
Queue: ['У_1', 'домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'У_1']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2']
Queue: ['римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2', 'римського_3']
Queue: ['патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'В

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [7]:
tree[0]

OrderedDict([('id', 1),
             ('form', 'У'),
             ('lemma', 'у'),
             ('upostag', 'ADP'),
             ('xpostag', 'Spsl'),
             ('feats', OrderedDict([('Case', 'Loc')])),
             ('head', 2),
             ('deprel', 'case'),
             ('deps', [('case', 2)]),
             ('misc',
              OrderedDict([('Id', '0003'),
                           ('LTranslit', 'u'),
                           ('Translit', 'U')]))])

In [8]:
def extract_features(stack, queue):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
        if stack_top["xpostag"]:
            features["s0-xtag"] = stack_top["xpostag"]
        if stack_top["feats"]:
            for k, v in stack_top["feats"].items():
                features["s0-" + k] = v
    if len(stack) > 1:
        stack_top = stack[-2]
        features["s1-word"] = stack_top["form"]
        features["s1-lemma"] = stack_top["lemma"]
        features["s1-tag"] = stack_top["upostag"]
        if stack_top["xpostag"]:
            features["s1-xtag"] = stack_top["xpostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        if queue_top['xpostag']:
            features["q0-xtag"] = queue_top["xpostag"]
        if queue_top["feats"]:
            for k, v in queue_top["feats"].items():
                features["q0-" + k] = v
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
        if queue_next["xpostag"]:
            features["q1-xtag"] = queue_next["xpostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
        if queue[2]["xpostag"]:
            features["q2-xtag"] = queue[2]["xpostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
        if queue[3]["xpostag"]:
            features["q3-tag"] = queue[3]["xpostag"]
    if stack and queue:
        features["distance"] = queue[0]["id"] - stack[-1]["id"]
        features["same_tag"] = queue[0]["upostag"] == stack[-1]["upostag"]
    
    if len(stack) > 1 and len(queue) > 1:
        features["distance_1_1"] = queue[1]["id"] - stack[-2]["id"]
        features["same_tag_1_1"] = queue[1]["upostag"] == stack[-2]["upostag"]
        features["distance_1"] = queue[0]["id"] - stack[-2]["id"]
        features["same_tag_1"] = queue[0]["upostag"] == stack[-2]["upostag"]
        
    if len(stack) > 2 and len(queue) > 2:
        features["distance_2_2"] = queue[2]["id"] - stack[-3]["id"]
        features["same_tag_2_2"] = queue[2]["upostag"] == stack[-3]["upostag"]
        features["distance_2"] = queue[0]["id"] - stack[-3]["id"]
        features["same_tag_2"] = queue[0]["upostag"] == stack[-3]["upostag"]
    return features

## Prepare train and test data

In [9]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        features.append(extract_features(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [10]:
features, labels = get_data(tree)
print("Number of words:", len(tree))
print("Number of actions:", len(labels))
print("List of actions taken:", labels)
print("Features:")
for word in features:
    print(word)

Number of words: 14
Number of actions: 29
List of actions taken: ['shift', 'left', 'shift', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'left', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 'q0-word': 'У', 'q0-lemma': 'у', 'q0-tag': 'ADP', 'q0-xtag': 'Spsl', 'q0-Case': 'Loc', 'q1-word': 'домі', 'q1-tag': 'NOUN', 'q1-xtag': 'Ncmsln', 'q2-tag': 'ADJ', 'q2-xtag': 'Ao-msgf', 'q3-tag': 'Ncmsgy', 'distance': 1, 'same_tag': False}
{'s0-word': 'У', 's0-lemma': 'у', 's0-tag': 'ADP', 's0-xtag': 'Spsl', 's0-Case': 'Loc', 's1-word': 'ROOT', 's1-lemma': 'ROOT', 's1-tag': 'ROOT', 'q0-word': 'домі', 'q0-lemma': 'дім', 'q0-tag': 'NOUN', 'q0-xtag': 'Ncmsln', 'q0-Animacy': 'Inan', 'q0-Case': 'Loc', 'q0-Gender': 'Masc', 'q0-Number': 'Sing', 'q1-word': 'римського', 'q1-tag': 'ADJ', 'q1-xtag': 'Ao-msgf

In [11]:
train_features, train_labels = [], []
for tree in trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [12]:
# Test data

with open("uk_iu-ud-test.conllu", "r") as f:
    data = f.read()
test_trees = parse(data)

test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

35124 35124


## Train a classifier

In [13]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [14]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))


Total number of features:  147133


In [15]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

#print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

In [16]:
# lrc = SVC(verbose=1, max_iter=50, C=2, random_state=42, gamma=0.2)

In [17]:
lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial", max_iter=1000, verbose=1, C=1.1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 412 seconds


/Users/sasha/programming/edu/ucu/computer-vision/.env/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.9min finished


LogisticRegression(C=1.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=1,
                   warm_start=False)

In [18]:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.88      0.92      0.90      8658
      reduce       0.87      0.80      0.84      9350
       right       0.79      0.83      0.81      8291
       shift       0.88      0.89      0.89      8825

    accuracy                           0.86     35124
   macro avg       0.86      0.86      0.86     35124
weighted avg       0.86      0.86      0.86     35124



## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [19]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        elif not stack and queue:
            return sorted(relations)
        else:
            features = extract_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
#             if log:
#                 print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
#                 print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
#                 print("Relations:", relations)
#                 print(action)
#                 print("========================")
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [20]:
total, tp = 0, 0
for tree in test_trees:
    tree = [t for t in tree if type(t["id"])==int]
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, lrc, vec)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))

Total: 17116
Correctly defined: 12585
UAS: 0.74


### Results
Baseline: 0.71

First improvement: 0.74
